# Tarea 6: Integración con Taylor

Fecha de envío del PR inicial: **viernes 5 de mayo**

Fecha de aceptación del PR: **martes 16 de mayo, antes de la clase**

---

## Ejercicio 1

Usando su implementación de polinomios de Taylor, escriban un integrador para la ecuación diferencial que se desarrolló en este ejemplo, esto es, $\dot{x} = x^2$ con la condición inicial $x(0) = 3$. 

El integrador debe hacer las operaciones necesarias para obtener automáticamente los coeficientes $x_{[k]}$, *en cada paso de integración*, a partir de la condición inicial local. Un requisito básico para esto es que tengan una implementación de la función $P_\alpha(x) = [g(x)]^\alpha$ con $g(x)$ un polinomio de Taylor, que hicieron en la "Tarea5", y que funcione bien en particular para `alpha::Int`.

La implementación debe consistir de varias funciones: 

- Una función donde se calculen los coeficientes $x_{[k]}$ de la expansión. Esta función deberá llamar a otra donde se implementan las recurrencias que imponen las ecuaciones de movimiento.

- Una función donde se obtenga el paso de integración $h$ como se describió en el notebook 10.

- Otra función donde se haga la suma usando el método de Horner.

- Finalmente, una función que combine las funciones anteriores para hacer la integración desde un tiempo inicial a uno final. En este punto, *fingiremos ignorancia*, en el sentido de  que el tiempo inicial es cero, y el tiempo final será $0.5$ (que está más allá de donde la solución está definida).


Dado que conocemos la solución analítica de este problema, grafiquen como función de $t$ el error relativo de su integrador (respecto al valor del resultado analítico).

In [1]:
include("runtest_taylor.jl")

Test Summary:                      | Pass  Total
  pruebas de operaciones para Taylor |   19     19


Base.Test.DefaultTestSet("pruebas de operaciones para Taylor",Any[],19,false)

In [2]:
function integra_taylor(f,x0,epsi)
    
   #vector de coeficientes x_k 
    
    f1=copy(f.cofun)
    
    x=zeros(f1)
    x[1]=x0
    p=length(f1)
    
    for i=2:p
        x[i]=f1[i-1]/p

    end
  
    #Cálculo  de h
    
    if (epsi/f1[p])^(1/p)<(epsi/f1[p-1])^(1/p)
        h=(epsi/f1[p])^(1/p)
    else 
        h=(epsi/f1[p-1])^(1/p)
    end
    
    #Método de Horner. 
    #Código basado del que aparece en: https://en.wikipedia.org/wiki/Horner%27s_method
    
     sum = x[end]
     for k = length(x)-1:-1:1
       sum = x[k] + h*sum
     end
     
    
    return sum 
    
end   

integra_taylor (generic function with 1 method)

In [3]:
f=Taylor([0,0,1],2)

Ty.Taylor{Int64}([0,0,1],2)

In [5]:
integra_taylor(f,3,1e-12)

3.0

---

## Ejercicio 2

Repitan la integración del ejercicio anterior usando el método de Runge-Kutta de 4o orden con paso de integración fijo (que es lo más sofisticado que conocen hasta ahora) y comparen los resultados del error relativo con los obtenidos con el método de Taylor. En particular, finjan ignorancia de la misma manera que en el ejercicio anterior.

Escribimos el método de  Runge-Kutta de 4to orden para el oscilador armónico simple.

In [ ]:
function rk4(f,x0,t0,tf,h= 0.01)

imax = (tf-t0)/h #número máximo de iteraciones del proceso
ep=1e-6   # Tolerancia 
#x1=0    
#delta = 1.0 #valor inicial de delta    
    

#while (delta>e)
for i=0:imax
    
    k1=h*f1(x0,t0) 
    k2=h*f1(x0+k1/2,t0+h/2)
    k3=h*f1(x0+k2/2,t0+h/2)
    k4=h*f1(x0+k3/2,t0+h)
    
    x1=x0+(k1+2*k2+2*k3+k4)/6
        
   # delta = abs(x1-x0)#Nuevo valor de delta (debe estar antes de la actualización de x0)
    x0=x1   #Nuevo valor de la variable
    t0=t0+h #Actualización del tiempo
       
    return x1    
        
end
    
end   


In [ ]:
function f1(x,t)
    return x^2
end

In [ ]:
rk4(f1,3,0,1,1e-15)

---

## Ejercicio 3

Integra la ecuación de movimiento para el oscilador armónico, $\ddot{x} = -2x$, con $x(0)=2$, $\dot{x}(0)=0$, durante 10^4 periodos de oscilación (o sea, hasta $t_f = 10^4 \cdot 2\pi/\sqrt{2}$, usando el método de Taylor y el método de Runge-Kutta de 4o orden con paso de integración constante. Compara cómo cambia la energía (respecto al valor al tiempo cero) en función del tiempo en ambos métodos.

In [1]:
function rk42(f,g,x0,v0,t0,tf,h= 0.01)
    imax = (tf-t0)/h #número máximo de iteraciones del proceso

    for i=0:imax
    
        k11=h*f(x0,v0,t0)
        k12=h*g(x0,v0,t0)

        k21=h*f(x0+k11/2,v0+k12/2,t0+h/2)
        k22=h*g(x0+k11/2,v0+k12/2,t0+h/2)

        k31=h*f(x0+k21/2,v0+k22/2,t0+h/2)
        k32=h*g(x0+k21/2,v0+k22/2,t0+h/2)

        k41=h*f(x0+k31/2,v0+k32/2,t0+h)
        k42=h*g(x0+k31/2,v0+k32/2,t0+h)


        x1=x0+(k11+2*k21+2*k31+k41)/6
        v1=v0+(k12+2*k22+2*k32+k42)/6

        t0=t0+h#actualización del tiempo
        x0=x1 #actualización de las variables
        v0=v1
        
         return x0,v0
    end  
    
   
end   

rk42 (generic function with 2 methods)

In [2]:
function ff(x,v,t)
    return v
end

ff (generic function with 1 method)

In [3]:
function gg(x,v,t)
    return -2*x
end

gg (generic function with 1 method)

In [4]:
rk42(ff,gg,2,0,0,1)

(1.999833335,-0.039999)